## PARCEIROS 360 - FATURAMENTO

###  Pagamento - Utilização Factoring & Bloqueios

- <b>Analista(s)</b>: Daniel Rezende
- <b>Projeto</b>: Parceiros 360 - Um modelo de análise de risco para auxiliar no controle das tomadas de decisões através de análises preditivas
- <b>Data</b>: 24/10/2022
- <b>Descrição</b>:

In [37]:
!python --version

Python 3.8.16


### BIBLIOTECAS:

In [38]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## modelo
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from treeinterpreter import treeinterpreter as ti

## alertas
import warnings
warnings.filterwarnings("ignore")

### LAYOUT:

In [39]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

rc={'font.size': 16, 'axes.labelsize': 16, 'legend.fontsize': 16,
    'axes.titlesize': 18, 'xtick.labelsize': 16, 'ytick.labelsize': 16}

sns.set(rc=rc)
sns.set_style("darkgrid")
sns.set_palette("pastel")

### CONFIGS:

In [40]:
#pd.set_option('max_columns', 200)
pd.options.display.max_columns = 200 

seed = 42
np.random.seed(seed)

### FUNÇÕES

In [41]:
#SE PRECISAR DO ONE HOT USAR ANTES DO SCALERS PRA NAO DAR ERRO

def scalers(base):
    nova_base = base.copy()
    scaler = MinMaxScaler(feature_range=(0,1))
    for column in base.columns:
        aux = scaler.fit_transform(base[[column]])
        nova_base[column] = aux
    return nova_base

In [42]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET

def makedf(base_treino_scaled, dados, kpi_nome, base_treino):
    
    kmeans = KMeans(n_clusters = 3, max_iter = 300, init = 'k-means++', n_init = 10, random_state = 0)
    clusters = kmeans.fit_predict(base_treino_scaled)

    model = RandomForestClassifier(random_state=0)
    model.fit(base_treino_scaled,clusters)
    prediction, bias, contributions = ti.predict(model, base_treino_scaled)

    contributions = np.round(contributions,5)*1000

    contributions_c1 = []
    contributions_c2 = []
    contributions_c3 = []
    contributions_filtered = []

    for i in range(len(contributions)):
        contributions_c1.append([contributions[i][0][0], contributions[i][1][0], contributions[i][2][0], contributions[i][3][0], contributions[i][4][0]])
        contributions_c2.append([contributions[i][0][1], contributions[i][1][1], contributions[i][2][1], contributions[i][3][1], contributions[i][4][1]])
        contributions_c3.append([contributions[i][0][2], contributions[i][1][2], contributions[i][2][2], contributions[i][3][2], contributions[i][4][2]])
        
    for i in range(len(clusters)):
        if clusters[i] == 0:
            contributions_filtered.append(contributions_c1[i])
        if clusters[i] == 1:
            contributions_filtered.append(contributions_c2[i])
        if clusters[i] == 2:
            contributions_filtered.append(contributions_c3[i])

    scaler = MinMaxScaler(feature_range=(0,100))
    contributions_filtered_scaled = scaler.fit_transform(contributions_filtered)

    contributions_filtered = pd.DataFrame(contributions_filtered_scaled, columns=base_treino_scaled.columns + '_CONTRIBUTION')

    base_treino['KPI_original'] = dados[kpi_nome].values
    base_treino['resultado_modelo'] = clusters
    base_treino['resultado_modelo'] = base_treino['resultado_modelo'].map({0:'BOM', 1:'MEDIO', 2:'RUIM'})

    base_treino.reset_index(inplace=True, drop=False)

    results = pd.concat([base_treino, contributions_filtered], axis=1)
    return results

### DADOS:

In [43]:
PATH = "intermediate/"

***
#### FATURAMENTO: 

In [56]:
FILE = "etl_previsao_faturamento.xlsx"
etl_previsao_faturamento = pd.read_excel(PATH + FILE)
etl_previsao_faturamento['RAIZ_CNPJ'] = etl_previsao_faturamento['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
etl_previsao_faturamento = etl_previsao_faturamento.drop(['Unnamed: 0'], axis=1)
# etl_previsao_faturamento = etl_previsao_faturamento.drop(['Unnamed: 0', 'FORNECEDOR', 'AREA_ATUACAO'], axis=1)

etl_previsao_faturamento = etl_previsao_faturamento.drop_duplicates().reset_index(drop=True)

etl_previsao_faturamento

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO,ARRECADACAO_EFETIVA,AJUSTE,AGING_PGTO,PLANTA,INADIMPLENCIA,KPI_faturamento,ARRECADACAO_EFETIVA_oe,AJUSTE_oe,AGING_PGTO_oe,PLANTA_oe,INADIMPLENCIA_oe
0,00000000,2021-01-01,HODNET S/A,9200015,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_BOM,INADIMPLENCIA_BOM,MEDIO,0,0,1,0,0
1,00000000,2021-01-01,HODNET S/A,9200031,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_BOM,INADIMPLENCIA_BOM,MEDIO,0,0,1,0,0
2,00000000,2021-02-01,HODNET S/A,9200015,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_RUIM,INADIMPLENCIA_BOM,MEDIO,0,0,1,2,0
3,00000000,2021-02-01,HODNET S/A,9200031,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_RUIM,INADIMPLENCIA_BOM,MEDIO,0,0,1,2,0
4,00000000,2021-03-01,HODNET S/A,9200015,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_RUIM,INADIMPLENCIA_BOM,MEDIO,0,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34005,96534094,2022-07-01,ACCENTURE DO BRASIL LTDA,14025436,CONSULTORIA,ARRECADACAO_EFETIVA_MEDIO,AJUSTE_BOM,AGING_PGTO_RUIM,PLANTA_BOM,INADIMPLENCIA_RUIM,RUIM,1,0,2,0,2
34006,96534094,2022-07-01,ACCENTURE DO BRASIL LTDA,14062035,CONSULTORIA,ARRECADACAO_EFETIVA_MEDIO,AJUSTE_BOM,AGING_PGTO_RUIM,PLANTA_BOM,INADIMPLENCIA_RUIM,RUIM,1,0,2,0,2
34007,96534094,2022-07-01,ACCENTURE DO BRASIL LTDA,14060244,CONSULTORIA,ARRECADACAO_EFETIVA_MEDIO,AJUSTE_BOM,AGING_PGTO_RUIM,PLANTA_BOM,INADIMPLENCIA_RUIM,RUIM,1,0,2,0,2
34008,96534094,2022-07-01,ACCENTURE DO BRASIL LTDA,14024766,CONSULTORIA,ARRECADACAO_EFETIVA_MEDIO,AJUSTE_BOM,AGING_PGTO_RUIM,PLANTA_BOM,INADIMPLENCIA_RUIM,RUIM,1,0,2,0,2


In [57]:
etl_previsao_faturamento[etl_previsao_faturamento['RAIZ_CNPJ'] == '33067745']

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO,ARRECADACAO_EFETIVA,AJUSTE,AGING_PGTO,PLANTA,INADIMPLENCIA,KPI_faturamento,ARRECADACAO_EFETIVA_oe,AJUSTE_oe,AGING_PGTO_oe,PLANTA_oe,INADIMPLENCIA_oe
21403,33067745,2021-01-01,ERICSSON TELECOMUNICAÃÃES S/A,7000004,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_MEDIO,INADIMPLENCIA_BOM,MEDIO,0,0,1,1,0
21404,33067745,2021-01-01,ERICSSON TELECOMUNICAÃÃES S/A,14000049,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_MEDIO,INADIMPLENCIA_BOM,MEDIO,0,0,1,1,0
21405,33067745,2021-01-01,ERICSSON TELECOMUNICAÃÃES S/A,14004446,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_MEDIO,INADIMPLENCIA_BOM,MEDIO,0,0,1,1,0
21406,33067745,2021-02-01,ERICSSON TELECOMUNICAÃÃES S/A,7000004,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_MEDIO,INADIMPLENCIA_BOM,MEDIO,0,0,1,1,0
21407,33067745,2021-02-01,ERICSSON TELECOMUNICAÃÃES S/A,14000049,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_MEDIO,INADIMPLENCIA_BOM,MEDIO,0,0,1,1,0
21408,33067745,2021-02-01,ERICSSON TELECOMUNICAÃÃES S/A,14004446,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_MEDIO,INADIMPLENCIA_BOM,MEDIO,0,0,1,1,0
21409,33067745,2021-03-01,ERICSSON TELECOMUNICAÃÃES S/A,7000004,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_BOM,PLANTA_BOM,INADIMPLENCIA_BOM,BOM,0,0,0,0,0
21410,33067745,2021-03-01,ERICSSON TELECOMUNICAÃÃES S/A,14000049,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_BOM,PLANTA_BOM,INADIMPLENCIA_BOM,BOM,0,0,0,0,0
21411,33067745,2021-03-01,ERICSSON TELECOMUNICAÃÃES S/A,14004446,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_BOM,PLANTA_BOM,INADIMPLENCIA_BOM,BOM,0,0,0,0,0
21412,33067745,2021-04-01,ERICSSON TELECOMUNICAÃÃES S/A,7000004,TELECOMUNICAÃÃES,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_MEDIO,INADIMPLENCIA_BOM,MEDIO,0,0,1,1,0


In [46]:
dados = etl_previsao_faturamento.copy()
dados.set_index(['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], inplace=True)
dados

ARRECADACAO_EFETIVA      AJUSTE        AGING_PGTO       PLANTA        INADIMPLENCIA KPI_faturamento  ARRECADACAO_EFETIVA_oe  AJUSTE_oe  AGING_PGTO_oe  PLANTA_oe  INADIMPLENCIA_oe
RAIZ_CNPJ COMPETENCIA NOME_FORNECEDOR                                                                                                                                                                                                   
0         2021-01-01  HODNET S/A                  ARRECADACAO_EFETIVA_BOM  AJUSTE_BOM  AGING_PGTO_MEDIO   PLANTA_BOM    INADIMPLENCIA_BOM           MEDIO                       0          0              1          0                 0
          2021-02-01  HODNET S/A                  ARRECADACAO_EFETIVA_BOM  AJUSTE_BOM  AGING_PGTO_MEDIO  PLANTA_RUIM    INADIMPLENCIA_BOM           MEDIO                       0          0              1          2                 0
          2021-03-01  HODNET S/A                  ARRECADACAO_EFETIVA_BOM  AJUSTE_BOM  AGING_PGTO_MEDIO  PLANTA_RUIM    INADIMPLENCIA_BOM           MEDIO                       0          0              1          2                 0
          2021-04-01  HODNET S/A                  ARRECADACAO_EFETIVA_BOM  AJUSTE_BOM  AGING_PGTO_MEDIO  PLANTA_RUIM    INADIMPLENCIA_BOM           MEDIO                       0          0              1          2                 0
          2021-05-01  HODNET S/A                  ARRECADACAO_EFETIVA_BOM  AJUSTE_BOM  AGING_PGTO_MEDIO  PLANTA_RUIM    INADIMPLENCIA_BOM           MEDIO                       0          0              1          2                 0
...                                                                   ...         ...               ...          ...                  ...             ...                     ...        ...            ...        ...               ...
96534094  2022-03-01  ACCENTURE DO BRASIL LTDA    ARRECADACAO_EFETIVA_BOM  AJUSTE_BOM   AGING_PGTO_RUIM  PLANTA_RUIM  INADIMPLENCIA_MEDIO           MEDIO                       0          0              2          2                 1
          2022-04-01  ACCENTURE DO BRASIL LTDA    ARRECADACAO_EFETIVA_BOM  AJUSTE_BOM   AGING_PGTO_RUIM  PLANTA_RUIM  INADIMPLENCIA_MEDIO           MEDIO                       0          0              2          2                 1
          2022-05-01  ACCENTURE DO BRASIL LTDA    ARRECADACAO_EFETIVA_BOM  AJUSTE_BOM   AGING_PGTO_RUIM   PLANTA_BOM  INADIMPLENCIA_MEDIO           MEDIO                       0          0              2          0                 1
          2022-06-01  ACCENTURE DO BRASIL LTDA    ARRECADACAO_EFETIVA_BOM  AJUSTE_BOM   AGING_PGTO_RUIM  PLANTA_RUIM  INADIMPLENCIA_MEDIO           MEDIO                       0          0              2          2                 1
          2022-07-01  ACCENTURE DO BRASIL LTDA  ARRECADACAO_EFETIVA_MEDIO  AJUSTE_BOM   AGING_PGTO_RUIM   PLANTA_BOM   INADIMPLENCIA_RUIM            RUIM                       1          0              2          0                 2

[8338 rows x 11 columns]

In [47]:
base_treino_kpi = dados[['ARRECADACAO_EFETIVA_oe', 'AJUSTE_oe', 'AGING_PGTO_oe', 'PLANTA_oe', 'INADIMPLENCIA_oe']]

In [48]:
base_treino_kpi_scaled = scalers(base_treino_kpi)

base_treino_kpi_scaled

ARRECADACAO_EFETIVA_oe  AJUSTE_oe  AGING_PGTO_oe  PLANTA_oe  INADIMPLENCIA_oe
RAIZ_CNPJ COMPETENCIA NOME_FORNECEDOR                                                                                        
0         2021-01-01  HODNET S/A                                   0.0        0.0       0.333333   0.000000               0.0
          2021-02-01  HODNET S/A                                   0.0        0.0       0.333333   0.666667               0.0
          2021-03-01  HODNET S/A                                   0.0        0.0       0.333333   0.666667               0.0
          2021-04-01  HODNET S/A                                   0.0        0.0       0.333333   0.666667               0.0
          2021-05-01  HODNET S/A                                   0.0        0.0       0.333333   0.666667               0.0
...                                                                ...        ...            ...        ...               ...
96534094  2022-03-01  ACCENTURE DO BRASIL LTDA                     0.0        0.0       0.666667   0.666667               0.5
          2022-04-01  ACCENTURE DO BRASIL LTDA                     0.0        0.0       0.666667   0.666667               0.5
          2022-05-01  ACCENTURE DO BRASIL LTDA                     0.0        0.0       0.666667   0.000000               0.5
          2022-06-01  ACCENTURE DO BRASIL LTDA                     0.0        0.0       0.666667   0.666667               0.5
          2022-07-01  ACCENTURE DO BRASIL LTDA                     0.5        0.0       0.666667   0.000000               1.0

[8338 rows x 5 columns]

***
### CLUSTERING COM K-Means:

In [49]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI, scaled
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET
#base_treino_kpi = BASE DE TREINO, SEM A COLUNA DO KPI


results_kpi = makedf(base_treino_kpi_scaled, dados, 'KPI_faturamento', base_treino_kpi)
results_kpi

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,ARRECADACAO_EFETIVA_oe,AJUSTE_oe,AGING_PGTO_oe,PLANTA_oe,INADIMPLENCIA_oe,KPI_original,resultado_modelo,ARRECADACAO_EFETIVA_oe_CONTRIBUTION,AJUSTE_oe_CONTRIBUTION,AGING_PGTO_oe_CONTRIBUTION,PLANTA_oe_CONTRIBUTION,INADIMPLENCIA_oe_CONTRIBUTION
0,0,2021-01-01,HODNET S/A,0,0,1,0,0,MEDIO,MEDIO,49.723715,87.624035,24.443186,13.534443,53.524836
1,0,2021-02-01,HODNET S/A,0,0,1,2,0,MEDIO,MEDIO,49.554873,85.862628,24.732603,15.577810,53.236969
2,0,2021-03-01,HODNET S/A,0,0,1,2,0,MEDIO,MEDIO,49.554873,85.862628,24.732603,15.577810,53.236969
3,0,2021-04-01,HODNET S/A,0,0,1,2,0,MEDIO,MEDIO,49.554873,85.862628,24.732603,15.577810,53.236969
4,0,2021-05-01,HODNET S/A,0,0,1,2,0,MEDIO,MEDIO,49.554873,85.862628,24.732603,15.577810,53.236969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8333,96534094,2022-03-01,ACCENTURE DO BRASIL LTDA,0,0,2,2,1,MEDIO,MEDIO,62.417498,65.964980,52.327922,9.878912,90.996509
8334,96534094,2022-04-01,ACCENTURE DO BRASIL LTDA,0,0,2,2,1,MEDIO,MEDIO,62.417498,65.964980,52.327922,9.878912,90.996509
8335,96534094,2022-05-01,ACCENTURE DO BRASIL LTDA,0,0,2,0,1,MEDIO,MEDIO,57.344589,65.069400,60.472505,11.329156,77.828137
8336,96534094,2022-06-01,ACCENTURE DO BRASIL LTDA,0,0,2,2,1,MEDIO,MEDIO,62.417498,65.964980,52.327922,9.878912,90.996509


In [50]:
results_kpi = results_kpi.drop_duplicates().reset_index(drop=True)
results_kpi

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,ARRECADACAO_EFETIVA_oe,AJUSTE_oe,AGING_PGTO_oe,PLANTA_oe,INADIMPLENCIA_oe,KPI_original,resultado_modelo,ARRECADACAO_EFETIVA_oe_CONTRIBUTION,AJUSTE_oe_CONTRIBUTION,AGING_PGTO_oe_CONTRIBUTION,PLANTA_oe_CONTRIBUTION,INADIMPLENCIA_oe_CONTRIBUTION
0,0,2021-01-01,HODNET S/A,0,0,1,0,0,MEDIO,MEDIO,49.723715,87.624035,24.443186,13.534443,53.524836
1,0,2021-02-01,HODNET S/A,0,0,1,2,0,MEDIO,MEDIO,49.554873,85.862628,24.732603,15.577810,53.236969
2,0,2021-03-01,HODNET S/A,0,0,1,2,0,MEDIO,MEDIO,49.554873,85.862628,24.732603,15.577810,53.236969
3,0,2021-04-01,HODNET S/A,0,0,1,2,0,MEDIO,MEDIO,49.554873,85.862628,24.732603,15.577810,53.236969
4,0,2021-05-01,HODNET S/A,0,0,1,2,0,MEDIO,MEDIO,49.554873,85.862628,24.732603,15.577810,53.236969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8333,96534094,2022-03-01,ACCENTURE DO BRASIL LTDA,0,0,2,2,1,MEDIO,MEDIO,62.417498,65.964980,52.327922,9.878912,90.996509
8334,96534094,2022-04-01,ACCENTURE DO BRASIL LTDA,0,0,2,2,1,MEDIO,MEDIO,62.417498,65.964980,52.327922,9.878912,90.996509
8335,96534094,2022-05-01,ACCENTURE DO BRASIL LTDA,0,0,2,0,1,MEDIO,MEDIO,57.344589,65.069400,60.472505,11.329156,77.828137
8336,96534094,2022-06-01,ACCENTURE DO BRASIL LTDA,0,0,2,2,1,MEDIO,MEDIO,62.417498,65.964980,52.327922,9.878912,90.996509


In [51]:
results_kpi.to_excel(r'outputs/previsao_faturamento.xlsx')